In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from openquake.hazardlib.geo.nodalplane import NodalPlane
from openquake.hazardlib.pmf import PMF
from openquake.hazardlib.geo.point import Point
from openquake.hazardlib.geo.line import Line
from openquake.hazardlib.scalerel import StrasserIntraslab, PeerMSR
from openquake.hazardlib.mfd import (EvenlyDiscretizedMFD, TruncatedGRMFD,
                                     YoungsCoppersmith1985MFD, ArbitraryMFD)
import inslab_builder as isb

## Construct an Inslab Source Model from a Slab 1.0 Interface Definition

In this demonstration we will construct a subduction in-slab source model based on the Slab 1.0 Sumatra subduction interface definition.

The subduction source will contain the following properties:

* Slab thickenss = 15.0 km

* Sources spaced every 5 km (sometimes wider spacing is used for images)

* Maximum Depth of the seismogenic interface is taken as 100.0 km, maximum seismogenic depth of the ruptures is 150 km

* Rupture mechanisms will be aligned such that:
        
    i. In the upper 40 km strike is aligned with the trench, dip is 15 degrees steeper than the interface and mechanism is reverse (essentially a plate flexure rupture)
      
    ii. In the 40 km - 80 km depth range strike is also aligned with the trench and dip made steeper to 30 degrees with respect to the interface. The mechanism is now extensional (gravity pull extension)
        
    iii. Below 80 km we consider two types of mechanism. In the first case the rupture plane is still aligned with the trench but the dip steeper (now 45 $^{\circ}$ with respect to the interface) and the mechanism still extension. This is given a probability of 0.6. In the second case the strike is perpendicular to the trench and the deep made steeper (to 60$^{\circ}$ with respect to the interface). The mechanism is now strike-slip , indicating a lateral stress lower slab flexure (Aegean-style deep mechanism) and is given a probability of 0.4.
     
* In the shallowest 80 km ruptures will be constrained by the slab geometry (impermeable), below 80 km the geometry is considered to be permeable.

* The magnitude frequency distribution will be a Truncated Gutenberg-Richter model with a = 5.0, b = 1.0, Mmin = 5.5 and Mmax = 8.0

* Two layers of points will be used (at 0.3 and 0.7 of the slab thickness)

* An aspect ratio of 1.0 is assumed (this will break as ruptures hit the slab thickness)

* The Inslab scaling model of Strasser et al. (2010) is used

#### Read in Data and Instantiate Model Builder

In [ ]:
top_edge_file = "./slab_demo_data/sum_top.in"
edge_contour_file = "./slab_demo_data/sum_contours.in"
spacing = 10.0 # km
maximum_interface_depth = 100.0 # km

model1 = isb.InSlabSourceBuilder.from_contour_files(top_edge_file,
                                                    edge_contour_file,
                                                    maximum_interface_depth,
                                                    spacing)

##### Visualise the interface

In [ ]:

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')
ax.plot_wireframe(model1.interface.mesh.lons,
           model1.interface.mesh.lats,
           -model1.interface.mesh.depths,
           color="b")


ax.set_xlabel(r"Longitude $^{\circ}$")
ax.set_ylabel(r"Latitude $^{\circ}$")
ax.set_zlabel("Depth (km)")
ax.set_zlim(-140., 0.0)

ax.set_zticks([0.0, -20, -40, -60, -80, -100, -120, -140])
ax.set_zticklabels(["0", "20", "40", "60", "80", "100", "120", "140"])

#### Render the sources normal to the interface

In [ ]:
thickness = 15.0 # km
layers = [0.3, 0.7] # Two layers

# Format is [((upper_depth_1, lower_depth_1), NodalPlane(strike_1, dip_1, rake_1)),
#            ((upper_depth_2, lower_depth_2), NodalPlane(strike_2, dip_2, rake_2))
#             ...
#            ((upper_depth_n, lower_depth_n), NodalPlane(strike_n, dip_n, rake_n))]

mechanism_distribution = [((0.0, 40.0), PMF([(1.0, NodalPlane(0.0, 15.0, 90.0))])),
                          ((40.0, 80.0), PMF([(1.0, NodalPlane(0.0, 30.0, -90.0))])),
                          ((80.0, np.inf), PMF([(0.6, NodalPlane(0.0, 45.0, -90.0)),
                                                (0.4, NodalPlane(90.0, 60.0, 0.0))]))
                           ]

mechanism_type = "Relative"

# Porosity distribution
# Format is [((upper_depth_1, lower_depth_1), False),
#            ((upper_depth_2, lower_depth_2), False),
#           ... 
#            (upper_depth_n, lower_depth_n), False)]
porosity = [((0.0, 80.0), False),
            ((80.0, np.inf), True)]

model1.render_inslab_points_normal(thickness,
                                   depth_npd=mechanism_distribution,
                                   layer_fractions=layers,
                                   porosity_distribution=porosity,
                                   npd_type=mechanism_type)


In [ ]:
# Visualise the interface and lower surface
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')
ax.plot_wireframe(model1.interface.mesh.lons,
           model1.interface.mesh.lats,
           -model1.interface.mesh.depths,
           color="b")
ax.plot_wireframe(model1.lower_surface.lons,
           model1.lower_surface.lats,
           -model1.lower_surface.depths,
           color="r")

ax.set_xlabel(r"Longitude $^{\circ}$")
ax.set_ylabel(r"Latitude $^{\circ}$")
ax.set_zlabel("Depth (km)")
ax.set_zlim(-140., 0.0)

ax.set_zticks([0.0, -20, -40, -60, -80, -100, -120, -140])
ax.set_zticklabels(["0", "20", "40", "60", "80", "100", "120", "140"])

In [ ]:
# Get the inslab points
inslab_pnts = np.array(
    [(pnt.longitude, pnt.latitude, pnt.depth) for pnt in model1.inslab_points]
    )

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')
# Plot the interface
ax.plot_wireframe(model1.interface.mesh.lons,
           model1.interface.mesh.lats,
           -model1.interface.mesh.depths,
           color="b")
# Plot the lower surface
ax.plot_wireframe(model1.lower_surface.lons,
           model1.lower_surface.lats,
           -model1.lower_surface.depths,
           color="r")
# Plot the source anchor points
ax.scatter(inslab_pnts[:, 0], inslab_pnts[:, 1], -inslab_pnts[:, 2], ".", color="k")

ax.set_xlabel(r"Longitude $^{\circ}$")
ax.set_ylabel(r"Latitude $^{\circ}$")
ax.set_zlabel("Depth (km)")
ax.set_zlim(-140., 0.0)
ax.set_zticks([0.0, -20, -40, -60, -80, -100, -120, -140])
ax.set_zticklabels(["0", "20", "40", "60", "80", "100", "120", "140"])

#### Build the In-Slab Source Model (may take approx. 20 - 30 seconds)

In [ ]:
mfd = TruncatedGRMFD(min_mag=5.5, max_mag=8.0, bin_width=0.1, a_val=5.0, b_val=1.0)

model1.build_source_model(mfd,
                          "SUM_IS_MOD", # Stem for all IDs
                          msr=StrasserIntraslab(),
                          usd=0.0,  # Master upper seismogenic depth (no ruptures exceed this even if permeable)
                          lsd=150.0, # Master lower seismogenic depth (no ruptures exceed this even if permeable)
                          aspect=1.0)

#### Export the Source Model to file (may also take some time depending on number of sources)

In [ ]:
output_file = "./output_models/Example_Sumatra_Inslab1.xml"
name_of_source_model = "Sumatra Inslab Model Version 1.0"
model1.write_source_model(output_filename=output_file,
                          source_model_name=name_of_source_model)

## PEER Test Example

The second, and perhaps simpler, example demonstrates how to construct the in-slab model adopted within the PEER tests.

The particulars of the model are:

1. Slab thickness 12.5 km

2. Rupture Mechanism - All ruptures striking parallel to the interface, with with a dip of 35$^{\circ}$ relative to the interface. Reverse faulting is assumed here, though not relevent for the actual PEER test.

3. All depths are assumed impermeable

4. Two layers are used [0.3, 0.7]

5. The PEER scaling relation is used

6. The MFD is assumed to be a Truncated Gutenberg Richter with a = 2.1139, b = 1.0, bin_width=0.1, Mmin = 5.0, Mmax = 8.0

7. Aspect ratio is assumed to be 1.5

##### Build the surface from three parallel edges

In [ ]:
top_edge = Line([Point(-65.50625, -0.44967, 25.0),
                 Point(-65.50625, 0.44967, 25.0)])
middle_edge = Line([Point(-65.0000, -0.44967, 57.5),
                    Point(-65.0000, 0.44967, 57.5)])
bottom_edge = Line([Point(-64.58666, -0.44967, 103.46),
                    Point(-64.58666, 0.44967, 103.46)])

spacing = 5.0

model2 = isb.InSlabSourceBuilder([top_edge, middle_edge, bottom_edge], spacing)

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')

ax.plot_wireframe(model2.interface.mesh.lons,
           model2.interface.mesh.lats,
           -model2.interface.mesh.depths,
           color="b")


ax.scatter(model2.interface.mesh.lons,
           model2.interface.mesh.lats,
           -model2.interface.mesh.depths,
           marker='.')

ax.set_xlabel(r"Longitude $^{\circ}$")
ax.set_ylabel(r"Latitude $^{\circ}$")
ax.set_zlabel("Depth (km)")
ax.set_zlim(-120., 0.0)
ax.set_zticks([0.0, -20, -40, -60, -80, -100, -120])
ax.set_zticklabels(["0", "20", "40", "60", "80", "100", "120"])

In [ ]:
# Build the in-slab source configuration
thickness = 12.5
depth_npd = [
    ((0., np.inf), PMF([(1.0, NodalPlane(0., 35., 90))]))
]
porosity_dist = [((0., np.inf), False)]

layers = [0.3, 0.7]

#mfd = TruncatedGRMFD(5.0, 8.0, 0.01, 2.1139, 0.8)

model2.render_inslab_points_normal(thickness, depth_npd, layers, porosity_dist, npd_type="Relative")

In [ ]:
# Get the inslab points
inslab_pnts = np.array(
    [(pnt.longitude, pnt.latitude, pnt.depth) for pnt in model2.inslab_points]
    )

# Visualise the configuration
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')

ax.plot_wireframe(model2.interface.mesh.lons,
           model2.interface.mesh.lats,
           -model2.interface.mesh.depths,
           color="b")
ax.plot_wireframe(model2.lower_surface.lons,
           model2.lower_surface.lats,
           -model2.lower_surface.depths,
           color="r")
ax.scatter(inslab_pnts[:, 0], inslab_pnts[:, 1], -inslab_pnts[:, 2], marker='.')

ax.set_xlabel(r"Longitude $^{\circ}$")
ax.set_ylabel(r"Latitude $^{\circ}$")
ax.set_zlabel("Depth (km)")
ax.set_zlim(-120., 0.0)
ax.set_zticks([0.0, -20, -40, -60, -80, -100, -120])
ax.set_zticklabels(["0", "20", "40", "60", "80", "100", "120"])

In [ ]:
# Build the source model
mfd = TruncatedGRMFD(min_mag=5.0, max_mag=8.0, bin_width=0.1, a_val=2.1139, b_val=0.8)

model2.build_source_model(mfd,
                          "PEER_IS_TEST",
                          msr=PeerMSR(),
                          usd=0.0,
                          lsd=120.0,
                          aspect=1.5)

In [ ]:
# Export the source model
output_file = "./output_models/PEER_Test_Example1.xml"
source_name = "PEER Test Inslab Example 1"

model2.write_source_model(output_file, source_name)